In [1]:
pip install evaluate rouge rouge_score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c4bc7efe68b8b0ea21ee2bebbdcd25d538e914f0f536a935fb0ff81c27bedc2f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [2]:
pip install -U datasets fsspec huggingface-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 33.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.31.1
    Uninstalling huggingface-hub-0.31.1:
      Successfully uninstalled huggingface-hub-0.31.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.

In [3]:
import os
import numpy as np
import torch
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

os.environ["WANDB_DISABLED"] = "true"

In [4]:
MODEL_NAME = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(MODEL_NAME)
model     = MarianMTModel.from_pretrained(MODEL_NAME)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [5]:
# Load dataset
ds = load_dataset("ncduy/mt-en-vi")

if "validation" not in ds:
    ds["validation"] = ds["test"]

# Preprocess
def preprocess_fn(ex):
    src = ex["en"]
    tgt = ex["vi"]
    inputs = tokenizer(src, max_length=64, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt, max_length=64, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_ds = ds.map(preprocess_fn, batched=True, num_proc=8, remove_columns=ds["train"].column_names)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

README.md:   0%|          | 0.00/3.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/597M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=8):   0%|          | 0/11316 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=8):   0%|          | 0/11225 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [6]:
# Metrics
bleu  = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bleu_res  = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_res = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {
        "bleu":   bleu_res["score"],
        "rouge1": rouge_res["rouge1"],
        "rouge2": rouge_res["rouge2"],
        "rougeL": rouge_res["rougeL"],
    }

In [7]:
training_args = Seq2SeqTrainingArguments(
    output_dir               = "./marianMT-finetuned",
    per_device_train_batch_size = 256,
    per_device_eval_batch_size  = 256,
    num_train_epochs         = 3,
    learning_rate            = 3e-5,
    eval_strategy             = "epoch",
    save_strategy            = "epoch",
    predict_with_generate    = True,
    save_total_limit         = 1,
    logging_dir              = "./logs",
    logging_strategy         = "epoch",
    bf16                     = True,   # bật nếu GPU hỗ trợ
)

trainer = Seq2SeqTrainer(
    model            = model,
    args             = training_args,
    train_dataset    = tokenized_ds["train"],
    eval_dataset     = tokenized_ds["validation"],
    tokenizer        = tokenizer,
    data_collator    = data_collator,
    compute_metrics  = compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-7-f59550ab4c70>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel
1,1.676500,1.304628,41.668980,0.744386,0.556162,0.686283
2,1.521000,1.256452,42.307810,0.746786,0.560146,0.689297
3,1.474900,1.244196,42.473410,0.747123,0.561114,0.689991


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=33804, training_loss=1.5574964316353095, metrics={'train_runtime': 6585.667, 'train_samples_per_second': 1313.968, 'train_steps_per_second': 5.133, 'total_flos': 1.464798687610798e+17, 'train_loss': 1.5574964316353095, 'epoch': 3.0})

In [9]:
test_results = trainer.evaluate(eval_dataset=tokenized_ds["test"])
print(f"BLEU:   {test_results['eval_bleu']:.2f}")
print(f"ROUGE1: {test_results['eval_rouge1']:.4f}")
print(f"ROUGE2: {test_results['eval_rouge2']:.4f}")
print(f"ROUGEL: {test_results['eval_rougeL']:.4f}")


BLEU:   42.98
ROUGE1: 0.7477
ROUGE2: 0.5624
ROUGEL: 0.6911


In [13]:
devie = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def translate_batch_marian(src_texts, num_beams=5, max_length=64):
    batch = tokenizer(
        src_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to("cuda")

    generated_ids = model.generate(
        input_ids        = batch["input_ids"],
        attention_mask   = batch["attention_mask"],
        num_beams        = num_beams,
        max_length       = max_length,
        early_stopping   = True,
        no_repeat_ngram_size = 2,
        length_penalty   = 1.0,
    )
    return tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )

# 4. Ví dụ test
src_sentences = [
    "Hello, how are you?",
    "What is your name?",
    "Don't be pressure about life",
    "When the storm gone, you will never remember how you lived through it",
    "The weather today is nice."
]

translations = translate_batch_marian(src_sentences)
for src, tgt in zip(src_sentences, translations):
    print(f"{src}")
    print(f"{tgt}\n")

Hello, how are you?
Xin chào, cô khỏe không?

What is your name?
Tên anh là gì?

Don't be pressure about life
Đừng gây áp lực lên cuộc sống.

When the storm gone, you will never remember how you lived through it
Khi cơn bão tan, anh sẽ không bao giờ nhớ mình đã sống như thế nào.

The weather today is nice.
Thời tiết hôm nay thật đẹp.

